# Flotorch ADK - Basic FlotorchADKAgent with custom config Example

This notebook demonstrates how to create a simple chat agent using FlotorchADKAgent
Perfect for getting started with Flotorch AI development.

## Requirements
- FLOTORCH_API_KEY environment variable
- FLOTORCH_BASE_URL environment variable


In [ ]:
%pip install --pre flotorch[adk]


In [ ]:
# Setup and imports
import asyncio
import os
import sys
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
print("✓ Environment setup complete")


In [ ]:
# Flotorch ADK imports
from flotorch.adk.llm import FlotorchADKLLM
from flotorch.adk.agent import FlotorchADKAgent
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService
from google.adk import Runner
from google.genai import types

print("✓ Flotorch ADK imports successful")


In [ ]:
# Configuration
APP_NAME = "flotorch_memory_service_example"
USER_ID = "flotorch_user_123"

print(f"App Name: {APP_NAME}")
print(f"User ID: {USER_ID}")


In [ ]:
# Create Flotorch LLM model
model = FlotorchADKLLM(
    model_id="openai/gpt-4o-mini",
    api_key=os.getenv("FLOTORCH_API_KEY"),
    base_url=os.getenv("FLOTORCH_BASE_URL"),
)

print("✓ Flotorch LLM model created")


In [ ]:
# Optional: Custom config (alternative to hardcoded configuration)
config = {
    "name": "sample_agent",
    "description": "Example Flotorch agent with user friendly assistance",
    "systemPrompt": "you are a helpful assistant",
    "llm": {
        "callableName": "openai/gpt-4o-mini"
    }
}

print("✓ Custom config defined")


In [ ]:
# Create Flotorch agent
flotorch_adk_client = FlotorchADKAgent(
    agent_name="flotorch_assistant",
    enable_memory=False,
    config=config  # Use custom config or remove for hardcoded
)

agent = flotorch_adk_client.get_agent()
print("✓ Flotorch agent created")


In [ ]:
# Setup runner
from pickle import APPEND


session_service = InMemorySessionService()
runner = Runner(
    agent=agent,
    app_name= APP_NAME,
    session_service=session_service
)

print("✓ Runner setup complete")


In [ ]:
# Helper function to send messages
def run_single_turn(query, session_id, user_id):
    """Send message to Flotorch agent and get response."""
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
    
    return "Sorry, I couldn't process that."

print("✓ Helper function defined")


In [ ]:
# Create session and test the agent
async def test_agent():
    session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
    
    print(f"Flotorch agent ready! Session: {session.id}")
    
    # Test with a simple message
    test_message = "Hello! Can you introduce yourself?"
    response = run_single_turn(test_message, session.id, USER_ID)
    
    print(f"\nUser: {test_message}")
    print(f"Flotorch AI: {response}")
    
    return session

# Run the test
session = await test_agent()


## Summary

This notebook demonstrated:
- ✓ Setting up Flotorch ADK environment
- ✓ Creating a Flotorch LLM model
- ✓ Building a basic agent
- ✓ Setting up a runner for agent interaction
- ✓ Testing the agent with messages

